<a href="https://colab.research.google.com/github/gsaveri/SinGAN/blob/master/SinGan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#SinGAN Colab Notebook

This repo was originally forked from https://github.com/tamarott/SinGAN, I just added the colab notebook.

[Project](https://tamarott.github.io/SinGAN.htm) | [Arxiv](https://arxiv.org/pdf/1905.01164.pdf) | [CVF](http://openaccess.thecvf.com/content_ICCV_2019/papers/Shaham_SinGAN_Learning_a_Generative_Model_From_a_Single_Natural_Image_ICCV_2019_paper.pdf) | [Supplementary materials](https://openaccess.thecvf.com/content_ICCV_2019/supplemental/Shaham_SinGAN_Learning_a_ICCV_2019_supplemental.pdf) | [Talk (ICCV`19)](https://youtu.be/mdAcPe74tZI?t=3191) 


### Official pytorch implementation of the paper: "SinGAN: Learning a Generative Model from a Single Natural Image"
#### ICCV 2019 Best paper award (Marr prize)

The code will clone the repository into your local colab space, and train sinGan with an input image.

##Code

In [ ]:
import os
from google.colab import files

### Clone the repository

In [ ]:
!git clone https://github.com/gsaveri/SinGAN.git

In [ ]:
cd SinGAN

### Install dependencies

In [ ]:
!pip install -r requirements.txt

In [ ]:
# !pip install torch==1.4.0 
# !pip install torchvision==0.5.0

### Train on uploaded image

In [ ]:
# Load an input image
input = files.upload()
input = list(input.keys())[0]

!cp "{input}" /content/SinGAN/Input/Images/"{input}"
!rm "{input}"

In [ ]:
!python main_train.py --input_name "{input}"

###  Random samples
To generate random samples from any starting generation scale, please first train SinGAN model on the desired image (as described above), then run

In [ ]:
gen_start_scale = 5

In [ ]:
folder = input[0:len(input)-4]

In [ ]:
!python random_samples.py --input_name "{input}" --mode random_samples --gen_start_scale "{gen_start_scale}"

### Harmonization

In [ ]:
# Load paint reference image
harmonization_ref = files.upload()
harmonization_ref = list(input.keys())[0]

!cp "{harmonization_ref}" /content/SinGAN/Input/Harmonization/"{harmonization_ref}"
!rm "{harmonization_ref}"

In [ ]:
!python harmonization.py --input_name "{input}" --ref_name astro.png --harmonization_start_scale "{harmonization_ref}"

###Super resolution

In [ ]:
!python SR.py --input_name "{input}"

### Paint to image

In [ ]:
# Load paint reference image
paint_ref = files.upload()
paint_ref = list(input.keys())[0]

!cp "{paint_ref}" /content/SinGAN/Input/Paint/"{paint_ref}"
!rm "{paint_ref}"

In [ ]:
!python paint2image.py --input_name "{input}" --ref_name "{paint_ref}" --paint_start_scale 2

In [ ]:
# clean trainedModel
# !rm -r /content/SinGAN/Output/RandomSamples/"{folder}"
# !rm -r /content/SinGAN/TrainedModels/"{folder}"